In [1]:
import sympy as sp
import numpy as np
from sympy import diff
from IPython.display import display

In [2]:
from commons import Surface,MethodOfOrthonormalFrames,StructureEquations,SurfaceCurve

In [3]:
u,v = sp.symbols("u,v")
du,dv=sp.symbols("du,dv")
duAdv= sp.symbols("duAdv")

In [ ]:
class VectorField:
    def __init__(self,u,v):
        # u,v <=> a domain D
        self.u=u
        self.v=v
    
    def differentiate_along_curve(func,curve):
        s,u,v=curve[0]
        